In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns


from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
train = pd.read_csv('/Users/tinatsai/Documents/GitHub/kkbox-music-recommendation-challenge/input/train.csv')[:500]

songs = pd.read_csv('/Users/tinatsai/Documents/GitHub/kkbox-music-recommendation-challenge/input/songs.csv')
#print(songs.head())
members = pd.read_csv('/Users/tinatsai/Documents/GitHub/kkbox-music-recommendation-challenge/input/members.csv')
#print(members.head(20))
train=train.merge(members, left_on='msno', right_on='msno', how='inner')
train['teller']=1.0
print(train.shape,train.head())



In [ ]:
#skipped songs get rating 1, listened songs get rating 5
train['rating']=train['target']*5-1
train.fillna(value=0,axis=1,inplace=True)

topusers=train.groupby(by=['msno'])['rating'].sum()
topsongs=train.groupby(by=['song_id'])['rating'].sum()
topsongs=topsongs.sort_values(0,ascending=False)   #[:20000]
print(topsongs)
#3.5M songs, we limit to top 30K songs

In [ ]:
def trans2vect(df,uid,pid,rate):
    from scipy.sparse import csr_matrix
    from sklearn.preprocessing import normalize
    toppid=train.groupby(by=[pid])[rate].sum()
    toppid=toppid.sort_values(0,ascending=False)   #[:20000]
    print(toppid)    
    #sparse matrix with product in rows and users in columns
    df=df[df[pid].isin(toppid.index)]
    user_u = list(df[uid].unique())
    song_u = list(toppid.index)
    col = df[uid].astype('category', category=user_u).cat.codes
    row = df[pid].astype('category', category=song_u).cat.codes
    songrating = csr_matrix((df[df[pid].isin(song_u)][rate].tolist(), (row,col)), shape=(len(song_u),len(user_u)))
    
    #normalize
    songrating_n = normalize(songrating, norm='l1', axis=0)
    return songrating_n,toppid.index

ratings,songU = trans2vect(train,'msno','song_id','teller')
ratings


In [ ]:
gender,genderU =trans2vect(train,'msno','gender','teller')
gender
from scipy.sparse import vstack
ratings=vstack((ratings,gender))
ratings

In [ ]:
bds,bdU =trans2vect(train,'msno','bd','teller')
bds
from scipy.sparse import vstack
ratings=vstack((ratings,bds))
ratings

In [ ]:
registered,registerU =trans2vect(train,'msno','registered_via','teller')
registered
from scipy.sparse import vstack
ratings=vstack((ratings,registered))
ratings

In [ ]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds       
#similarities = cosine_similarity(ratings)  #goes south with >15k songs
#print(similarities.shape)
#similarities

SongsU,Eigen,UsersU=svds(ratings, k=100)

#print(Eigen)
#print(SongsU)

In [ ]:
userU = list(train['msno'].unique())
index1=songU.append(genderU)
index2=index1.append(bdU)
index3=index2.append(registerU)
similarities = pd.DataFrame( cosine_similarity(SongsU,UsersU.T) , index=index3,columns=userU)

#find similar users
similarities.sort_values('T86YHdD4C9JSc274b1IlMkLuNdz4BQRB50fWWE7hx9g=',axis=1)

In [ ]:
#find top songs to recommend
similarities.sort_values('FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=',axis=0)[-10:]

In [ ]:
#find the third train record
similarities.loc[train.iloc[3]['song_id'],train.iloc[3]['msno']]

In [331]:
train['sim']=0.0
print(train.columns)

for xi in range(0,len(train)):
    train.iat[xi,14]=similarities.loc[train.iloc[xi]['song_id'],train.iloc[xi]['msno']]

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date', 'teller', 'rating', 'sim'],
      dtype='object')


In [332]:
print( train.groupby(by=['target'])['sim'].mean() )
print( train.groupby(by=['target'])['sim'].std() )
#split between 0 and 0.07

target
0   -0.004947
1    0.010104
Name: sim, dtype: float64
target
0    0.066114
1    0.132426
Name: sim, dtype: float64


In [333]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 21 20:41:10 2017
@author: eric
"""
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score



def sigmoid(z):
    g=1/(np.exp(-z)+1)
    return g


def cross_entropy(x,w,y):
    loss=0
    row=np.shape(y)[0]
    
    for i in range(0,row):
      if y[i]==1:
          loss=loss-np.log(sigmoid(np.dot(x[i],w)))
      else:
          loss=loss-np.log(sigmoid(1-np.dot(x[i],w)))
    
    loss=loss/row
    return loss

    
def gradient(x,w,y,l2reg,size,count):
    gradient_error=0
    row=np.shape(x)[0]
    
    if(size==row): # batch
        for i in range(0,row):
             pred=sigmoid(np.dot(x[i],w))
             gradient_error=gradient_error+(pred-y[i])*x[i]+(l2reg/row)*w
        gradient_error=gradient_error/row
    
    else:  # minibatch
        range_array=np.arange(0,row,size)
        for i in range(range_array[count],size+range_array[count]):
            if i >= row:
                size=(i-range_array[count])+1
                break
            else:
                pred=sigmoid(np.dot(x[i],w))
                gradient_error=gradient_error+(pred-y[i])*x[i]+(l2reg/row)*w
        gradient_error=gradient_error/size
    
    return gradient_error
 

class logistic_regression(object):
      def __init__(self,eta,l2reg=0,maxiter=1000,size=0):
          self.eta=eta
          self.iter=maxiter
          self.l2reg=l2reg #with l2 regularizer ,lambda coefficient in lagrange  mutiplier
          self.batch_size=size
          
      def fit(self,X,Y):
          ones = np.ones((np.shape(X)[0], 1))
          X= np.concatenate((ones, X), axis=1)
          row=np.shape(X)[0]
          #w = np.random.randn(np.shape(X)[1]+ 1)
          w=np.zeros(np.shape(X)[1])
          iter_last=self.iter
          costs_record = []
          if(self.batch_size==0):
            self.batch_size=row
         
          count=0
          w=w-self.eta* gradient(X,w,Y,self.l2reg,self.batch_size,count)
          for i in range(0,self.iter):
              count+=1
              if count*self.batch_size >= row:
                 count=0
              costs_record.append(cross_entropy(X,w,Y))
              #loss function is separate to regularizer but include in gradient(augmenting function)
              if np.all(gradient(X,w,Y,self.l2reg,self.batch_size,count) ==0):
                 iter_last=i
                 break
              w=w-self.eta*gradient(X,w,Y,self.l2reg,self.batch_size,count)
          
          plt.clf()
          #plt.plot(range(0,iter_last), costs_record)
          plt.show()
          self.w=w
          return self
          
      def predict(self,test_X):
          ones = np.ones((np.shape(test_X)[0], 1))
          test_X= np.concatenate((ones, test_X), axis=1)
          row=np.shape(test_X)[0]
          predict_y=[]
          #probability threshold :0.5
          for i in range(0,row):
              if sigmoid(np.dot(test_X[i],self.w))>0.5:
                 answer=1
              else:
                 answer=0
              predict_y.append(answer)
          predict_y=np.array(predict_y)
          return predict_y



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

test = pd.read_csv('/Users/tinatsai/Documents/GitHub/kkbox-music-recommendation-challenge/input/test.csv')[:500]
test=test.merge(members, left_on='msno', right_on='msno', how='inner')

train = train.fillna(-1)
test = test.fillna(-1)

cols = list(train.columns)
cols.remove('target')

for col in tqdm(cols):
    if train[col].dtype == 'object':
        train[col] = train[col].apply(str)
        test[col] = test[col].apply(str)

        le = LabelEncoder()
        train_vals = list(train[col].unique())
        test_vals = list(test[col].unique())
        le.fit(train_vals + test_vals)
        train[col] = le.transform(train[col])
        test[col] = le.transform(test[col])

        print(col + ': ' + str(len(train_vals)) + ', ' + str(len(test_vals)))

X = np.array(train.drop(['target','rating'], axis=1))
y = train['target'].values

X_test = np.array(test.drop(['id'], axis=1))
ids = test['id'].values

del train, test

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state = 12)

model=logistic_regression(0.01,l2reg=5,maxiter=100,size=10)
model.fit(X_train,y_train)
x_predict = model.predict(X_valid)

accuracy_score(y_valid, x_predict)

100%|██████████| 14/14 [00:00<00:00, 649.29it/s]
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in exp
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log


msno: 126, 193
song_id: 452, 457
source_system_tab: 7, 6
source_screen_name: 13, 14
source_type: 10, 10
gender: 3, 3


<Figure size 432x288 with 0 Axes>

ValueError: Found input variables with inconsistent numbers of samples: [50, 450]